# Credit cards customers EDA

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
sns.set_theme(style='whitegrid')
from scipy.stats import pearsonr
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')

In [ ]:
df.set_index('CLIENTNUM', inplace=True)
df = df[['Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']]

In [ ]:
df.shape

### Customer Age

In [ ]:
sns.histplot(data=df, x='Customer_Age')

In [ ]:
df['Customer_Age'].describe()

In [ ]:
df['Customer_Age_Years'] = (df['Customer_Age']/10).apply(math.floor)
df['Customer_Age_Years'] = pd.Categorical(df['Customer_Age_Years'].map({
    2: '20s', 3: '30s', 4: '40s', 5: '50s', 6: '60s', 7: '70s'
}), categories=['20s','30s','40s','50s','60s','70s'], ordered=True)
print(df['Customer_Age_Years'].value_counts())

sns.histplot(data=df, x='Customer_Age_Years', hue='Customer_Age_Years', discrete=True)

### Gender

In [ ]:
df['Gender'] = pd.Categorical(df['Gender'])
print(100.00 * df['Gender'].value_counts()/df.shape[0])
sns.histplot(data=df, x='Gender', hue='Gender')

### Dependent count

In [ ]:
sns.histplot(df['Dependent_count'])

In [ ]:
sns.scatterplot(data=df, y='Dependent_count', x='Customer_Age', hue=pd.Categorical(df['Dependent_count']))

In [ ]:
sns.histplot(data=df, x="Dependent_count", hue="Gender", multiple="dodge")

### Education level

In [ ]:
df['Education_Level'] = pd.Categorical(
    df['Education_Level'],
    categories=['Uneducated', 'High School', 'College', 'Graduate', 'Post-Graduate', 'Doctorate', 'Unknown'],
    ordered=True
)

In [ ]:
print(100.00 * df['Education_Level'].value_counts() / df.shape[0])
sns.histplot(data=df, y='Education_Level', hue='Education_Level')

### Marital status

In [ ]:
df['Marital_Status'] = pd.Categorical(df['Marital_Status'])
sns.histplot(data=df, x='Marital_Status', hue='Gender', multiple='dodge')

In [ ]:
100.00 * df['Marital_Status'].value_counts() / df.shape[0]

### Income

In [ ]:
df['Income_Category'].unique()

In [ ]:
df['Income_Category'] = pd.Categorical(
    df['Income_Category'],
    categories=['Less than $40K', '$40K - $60K', '$60K - $80K', '$80K - $120K', '$120K +', 'Unknown'],
    ordered=True
)

In [ ]:
sns.histplot(data=df, y='Income_Category', hue='Gender', multiple='stack')

In [ ]:
100.00 * df['Income_Category'].value_counts() / df.shape[0]

In [ ]:
sns.catplot(data=df, y='Income_Category', x='Credit_Limit', col='Card_Category', kind='strip')

### Card category

In [ ]:
df['Card_Category'] = pd.Categorical(df['Card_Category'], categories=['Blue', 'Silver', 'Gold', 'Platinum'], ordered=True)
print(100.00 * df['Card_Category'].value_counts() / df.shape[0])
sns.histplot(data=df, x='Card_Category', hue='Card_Category')

In [ ]:
g = sns.FacetGrid(df,col="Income_Category", hue='Card_Category', col_wrap=3, height=3)
g.map(sns.histplot, 'Card_Category')

### Credit Limit

In [ ]:
plot = sns.histplot(df['Credit_Limit'], kde=True)
plot.axvline(df['Credit_Limit'].mean(), 0,2000)
plot.annotate('Average value: '+str(round(df['Credit_Limit'].mean())), xy=(df['Credit_Limit'].mean()+500, 1000))

In [ ]:
sns.boxplot(data=df, y='Card_Category', x='Credit_Limit')

In [ ]:
sns.violinplot(data=df, y='Card_Category', x='Credit_Limit')

### Transaction amount

In [ ]:
sns.histplot(data=df, x='Total_Trans_Amt')

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=4)
model.fit(df[['Total_Trans_Amt']])
df['Total_Trans_Amt_Segment'] = pd.Categorical(model.predict(df[['Total_Trans_Amt']]))

In [ ]:
sns.histplot(data=df, x='Total_Trans_Amt', hue='Total_Trans_Amt_Segment')

In [ ]:
df[['Total_Trans_Amt_Segment', 'Total_Trans_Amt']].groupby('Total_Trans_Amt_Segment').agg({'Total_Trans_Amt': [np.min, np.mean, np.max]})

### Transaction count

In [ ]:
plot = sns.histplot(data=df, x='Total_Trans_Ct', kde=True)
plot.axvline(df['Total_Trans_Ct'].mean(), 0,2000)
plot.annotate('Average value: '+str(round(df['Total_Trans_Ct'].mean())), xy=(df['Total_Trans_Ct'].mean()-50, 700))

In [ ]:
sns.boxplot(data=df, x='Total_Trans_Ct', y='Card_Category')

### Utilization ratio

In [ ]:
plot = sns.histplot(data=df, x='Avg_Utilization_Ratio', kde=True)
plot.axvline(df['Avg_Utilization_Ratio'].mean(), 0,2000)
plot.annotate('Average value: '+str(round(df['Avg_Utilization_Ratio'].mean(), 3)), xy=(df['Avg_Utilization_Ratio'].mean()+0.02, 2000))

In [ ]:
df['Avg_Utilization_Ratio'].describe()

In [ ]:
sns.boxplot(data=df, x='Avg_Utilization_Ratio', y='Card_Category')

### Months on book

In [ ]:
sns.histplot(data=df, x='Months_on_book')

In [ ]:
df['Years_on_book'] = pd.Categorical((df['Months_on_book']/12).apply(math.floor), categories=[1, 2, 3, 4], ordered=True)

In [ ]:
sns.histplot(data=df, x='Years_on_book', discrete=True)

In [ ]:
sns.boxplot(data=df, x='Years_on_book', y='Customer_Age', hue='Gender')

In [ ]:
sns.boxplot(data=df, x='Years_on_book', y='Total_Trans_Ct')

In [ ]:
df[['Years_on_book', 'Total_Trans_Ct']].groupby('Years_on_book').mean()

### Total relationship count

In [ ]:
sns.histplot(data=df, x='Total_Relationship_Count')

### Months inactive

In [ ]:
sns.histplot(df['Months_Inactive_12_mon'], discrete=True)

In [ ]:
100.00 * df['Months_Inactive_12_mon'].value_counts() / df.shape[0]

In [ ]:
df['Months_Inactive_12_mon'] = pd.Categorical(df['Months_Inactive_12_mon'])
sns.boxplot(data=df, x='Months_Inactive_12_mon', y='Total_Revolving_Bal')

### Contact frequency

In [ ]:
sns.histplot(data=df, x='Contacts_Count_12_mon', discrete=True)

### Adhoc

In [ ]:
sns.boxplot(data=df, x='Total_Trans_Amt', y=pd.Categorical(df['Customer_Age_Years']))

In [ ]:
sns.boxplot(data=df, x='Customer_Age', y='Income_Category')

In [ ]:
sns.boxplot(data=df, x='Customer_Age', y='Card_Category')

In [ ]:
sns.pairplot(data=df)